In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr

# Getting data we need

In [20]:
# Function for reading in esm data
def get_esms_and_memebers_with_scenarios(directory, filename, scenarios):
    data = pd.read_csv(os.path.join(directory, filename))
    my_obj = data[(data['year'] == 2050) & 
        (data['acronym'] == 'NWN') &
        (data['experiment'].isin(scenarios))].groupby('ensemble')['experiment'].apply(lambda x: len(np.unique(x)))
    members = my_obj.keys()
    number_of_my_scenarios = my_obj.values
    indeces = np.where(number_of_my_scenarios == 4)[0]
    member = None
    if len(indeces) > 0:
        index = indeces[0]
        member = members[index]
    
    return data['esm'].values[0], member, data['variable'].values[0]

In [21]:
# Path to time series esm data
data_path = 'extracted_timeseries'
directory = os.path.join(data_path, 'land_regions_ts')

# The scenarios we want to be included
scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

esm_arr = [None] * len(os.listdir(directory))
member_arr = [None] * len(os.listdir(directory))
variable_arr = [None] * len(os.listdir(directory))
for i, filename in enumerate(os.listdir(directory)):
    try:
        esm, member, variable = get_esms_and_memebers_with_scenarios(directory, filename, scenarios)
        esm_arr[i] = esm
        member_arr[i] = member
        variable_arr[i] = variable
    except: 
        pass

# Convert data into pandas
full_collection = pd.DataFrame({'esm': esm_arr, 'ensemble': member_arr, 'variable': variable_arr})
full_collection = full_collection.dropna()

/var/folders/03/82f8dprj20zdn3zkgcd_p1280000gn/T/ipykernel_48815/1880694802.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  my_obj = data[(data['year'] == 2050) &
/var/folders/03/82f8dprj20zdn3zkgcd_p1280000gn/T/ipykernel_48815/1880694802.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  m

# Functions For Fitting Polynomial and Extracting Needed Data

In [5]:
# Get results for a given scenario (slash model)
def get_scenario_results(data, hist_data, ref_tas, experiment, variable, esm):
    # Different value column name for pr and tas
    value_column = 'ann_agg'
    if variable == 'pr':
        value_column = 'pr'

    # Get data specific to this scenario
    experiment_data = data[data['experiment'] == experiment].reset_index(drop=True)

    # Calculate tas anomaly from reference year
    experiment_data['ann_anomaly'] = experiment_data[value_column].copy() - ref_tas

    # print(f'{esm} {experiment} has {len(hist_data.year.values)} historical years and {len(experiment_data.year.values)} future years')

    # Fit data to 4th degree polynomial
    years = np.concatenate([hist_data.year.values, experiment_data.year.values])
    temps = np.concatenate([hist_data.ann_anomaly.values, experiment_data.ann_anomaly.values])
    coeffs = np.polyfit(years, temps, 4)
    # Evaluate with the found coefficients
    smooth_fit = np.polyval(coeffs, years)
    # Extract residuals
    residuals = temps - smooth_fit

    # Return dictionary of results
    return {'time_series': temps, 'fit': smooth_fit, 'residuals': residuals}

In [6]:
# General results for a given ESM
def get_model_results(esm, experiment_data, scenarios, variable):
    # Different value column name for pr and tas
    value_column = 'ann_agg'
    if variable == 'pr':
        value_column = 'pr'

    # Get model data
    data = experiment_data[experiment_data['esm'] == esm].reset_index(drop=True)

    # Get historical data
    hist_data = data[data['experiment'] == 'historical'].reset_index(drop=True)

    # avg tas over historic period
    ref_tas = hist_data[value_column].values.mean().copy()
    # ref_tas = hist_data[hist_data['year'] == 2014][value_column].values[0]

    # Calculating tas anomaly over the historical period
    hist_data['ann_anomaly'] = hist_data[value_column].copy() - ref_tas

    # Smooth polynomial fit over historic period
    coeffs = np.polyfit(hist_data['year'].values, hist_data['ann_anomaly'].values, 4)
    # Evaluate with the found coefficients
    smooth_fit = np.polyval(coeffs, hist_data['year'].values)

    # Model warming over historic period
    model_warming = smooth_fit[-1] - smooth_fit[0]

    # Get model weight
    # model_weight = 1 / (ref_warming + np.abs(model_warming - ref_warming))
    model_weight = 1 # Equal weighting for all models

    # Create model dictionary
    model_dict = {'esm': esm, 'reference_tas': ref_tas, 'weight': model_weight}

    # Get results for each scenario and add it to the model's dictionary
    for exp in scenarios:
        scenario_dict = get_scenario_results(data, hist_data, ref_tas, exp, variable, esm)
        model_dict[exp] = scenario_dict
    
    # Return model results as dictionary
    return model_dict

# Normalized Model Weights

In [7]:
def add_normalized_weights(full_dict):
    weight_sum = 0
    for esm, data in full_dict.items():
        weight_sum += data['weight']

    for esm, data in full_dict.items():
        data['weighted_weight'] = data['weight'] / weight_sum
    
    return full_dict

# Internal Variability
$$V=\sum\limits_{m}W_{m}\text{var}_{s,t}(\varepsilon_{m,s,t})$$

In [8]:
def get_internal_variability(full_dict, scenarios):
    V = 0
    for esm, data in full_dict.items():
        residuals = []
        for exp in scenarios:
            residuals = np.concatenate([residuals, data[exp]['residuals']])

        V += data['weighted_weight'] * np.var(residuals)
    
    return V

# Model Uncertainty
$$M(t)=\frac{1}{N_{s}}\sum\limits_{s}\text{var}_{m}^{W}(x_{m,s,t})$$

In [9]:
def weighted_variance(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    return variance

In [10]:
def get_model_uncertainty(full_dict, n_years, esms, scenarios):
    scenario_inner_vars = np.zeros((len(scenarios), n_years))
    j = 0
    for exp in scenarios:
        model_fits = np.zeros((len(esms), n_years))
        model_weights = np.zeros(len(esms))
        i = 0
        for esm, data in full_dict.items():
            model_fits[i, :] = data[exp]['fit']
            model_weights[i] = data['weighted_weight']
            i += 1
        scenario_inner_vars[j,:] = [weighted_variance(model_fits[:,t], model_weights) for t in range(n_years)]
        j += 1

    M = np.array([np.sum(scenario_inner_vars[:,t])/len(scenarios) for t in range(n_years)])
    return M

# Scenario Uncertainty
$$S(t)=\text{var}_{s}(\sum\limits_{m}W_{m}x_{m,s,t})$$

In [11]:
def get_scenario_uncertainty(full_dict, n_years, scenarios):
    scenario_inner_sums = np.zeros((len(scenarios), n_years))
    iteration = 0
    for exp in scenarios:
        inner_sum = [0] * n_years
        for esm, data in full_dict.items():
            inner_sum += data['weighted_weight'] * data[exp]['fit']
        scenario_inner_sums[iteration, :] = inner_sum
        iteration += 1
    S = np.array([np.var(scenario_inner_sums[:,t]) for t in range(n_years)])
    return S

# Functions for running and extracting results for given set

In [12]:
def read_and_filter(directory, file, scenarios, ensemble_member, ipcc_region):
    data = pd.read_csv(os.path.join(directory, file), index_col=False)
    esm = data['esm'].values[0]
    variable = data['variable'].values[0]
    ensemble_member = full_collection[(full_collection['esm'] == esm) & (full_collection['variable'] == variable)]['ensemble'].values[0]

    data = data[(data['ensemble'] == ensemble_member) & 
        (data['experiment'].isin(scenarios + ['historical'])) &
        (data['acronym'] == ipcc_region)].copy()
    return data

In [13]:
def get_results(esms, scenarios, ensemble_member, variable, ipcc_region):
    # Input files
    input_directory = 'extracted_timeseries/land_regions_ts'
    input_files = [f'IPCC_land_regions_{variable}_{esm}_timeseries_1980-2099.csv' for esm in esms]

    # Read in data
    full_data = pd.concat([read_and_filter(input_directory, file, scenarios, ensemble_member, ipcc_region) for file in input_files]).copy()

    # Years in data
    years = np.sort(np.unique(full_data['year'].values))
    n_years = len(years)

    # Get model fits, and weights
    full_dict = {}
    for esm in esms:
        full_dict[esm] = get_model_results(esm, full_data, scenarios, variable)

    # Get variation results
    full_dict = add_normalized_weights(full_dict).copy()
    V = get_internal_variability(full_dict, scenarios)
    M = get_model_uncertainty(full_dict, n_years, esms, scenarios)
    S = get_scenario_uncertainty(full_dict, n_years, scenarios)

    # Plot
    total = M + S + V
    region_name = full_data['name'].values[0]
    df = pd.DataFrame({'year': years, 'Model Uncertainty': M / total, 'Scenario Uncertainty': S / total, 'Inter-Annual Variability': np.repeat(V, n_years) / total})

    return df

# Running HS

In [18]:
# TODO: Decide what the behavior should be if you choose model/scenario/ensemble combo that doesn't exist

# Get list of all models in full collection
# Note that NorESM2-LM only has all scenarios for tas not pr. See via below command so we also drop
esms = np.unique(full_collection[full_collection['esm'] != 'NorESM2-LM'].esm.values)

# Choose ESMs
# From Projection Min
esms2 = ['ACCESS-ESM1-5', 'GFDL-ESM4', 'CAMS-CSM1-0', 'CMCC-ESM2', 'MRI-ESM2-0']
# From Coefficient Min
esms3 = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'MIROC6', 'MRI-ESM2-0']

# Choose Scenarios
scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

# Choose Ensemble Member
ensemble_member = 'r1i1p1f1'

# Choose variable (change b/t tas an pr)
variable = 'tas'

# This gives the names each of the IPCC regions
access_esm_data = pd.read_csv(os.path.join(data_path, 'land_regions_ts', 'IPCC_land_regions_pr_ACCESS-ESM1-5_timeseries_1980-2099.csv'))
ipcc_land_regions = np.unique(access_esm_data['acronym'].values)

In [19]:
output_data_path = 'hs_output_data'
variable = 'pr'
for iter, ipcc_region in enumerate(ipcc_land_regions):
    df1 = get_results(esms, scenarios, ensemble_member, variable, ipcc_region)
    df2 = get_results(esms2, scenarios, ensemble_member, variable, ipcc_region)
    df3 = get_results(esms3, scenarios, ensemble_member, variable, ipcc_region)

    df1.to_csv(os.path.join(output_data_path, f'{variable}_{ipcc_region}_full_collection.csv'), index=False)
    df2.to_csv(os.path.join(output_data_path, f'{variable}_{ipcc_region}_projection_set.csv'), index=False)
    df3.to_csv(os.path.join(output_data_path, f'{variable}_{ipcc_region}_coefficient_set.csv'), index=False)